Loading elasticsearch

In [1]:
from elasticsearch import Elasticsearch

# Adds the HTTP header 'Authorization: Basic <base64 username:password>'
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", "lxDO2mQDVQPrxE3FRWAy")
)

print(es.ping())

True


Importing data for assignments(1,2,3)

In [2]:
import pandas as pd
# df_q : data for questions
df_q = pd.read_csv('C:\\Users\\A\\Music\\repos\\Information-Retrieval\\data\\questions.csv').fillna("")

In [3]:
df_q.head(3)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,115145,967227,2009-05-31 16:20:08,CommonsWare,2022-10-29 23:14:28,https://commonsware.com,"Pennsylvania, United States","<p>Mark Murphy, founder of CommonsWare, writes...",269231,13193,...,2022-08-19 16:46:16,2022-08-19 16:46:16,"How can I fix ""unexpected element <queries> fo...",<android><android-gradle-plugin><android-manif...,7,1,30,,,CC BY-SA 4.0
1,85371,358430,2009-03-31 21:38:26,sehe,2022-10-29 15:03:44,,Netherlands,<p>I'm a developer at heart.</p>\n<p>I love C+...,34058,16475,...,2021-03-06 02:51:07,2021-03-06 02:51:07,C++20 behaviour breaking existing code with eq...,<c++><c++17><c++20><spaceship-operator>,1,4,21,,,CC BY-SA 4.0
2,23354,994260,2008-09-29 05:46:02,Marc Gravell,2022-10-29 22:46:04,http://blog.marcgravell.com,"Forest of Dean, United Kingdom",<p>e-mail: marc.gravell@gmail.com<br/>\nblog: ...,320609,27181,...,2021-04-27 19:04:08,2022-04-25 07:56:58,Why does a zero-length stackalloc make the C# ...,<c#><stackalloc>,2,0,7,,,CC BY-SA 4.0


Indexing data(questions) based on the assignment

In [5]:
mapping = {
     "properties": {
            "ID": {"type": "integer"},
            "Body": {"type": "text"},
            "Tags": {"type": "text"}
         }}

es.indices.create(index="questions",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'questions'})

In [6]:
for i,row in df_q.iterrows():
  doc = {
      "ID" : row['Id.1'], # id.1=posts.id
      "Body" : row["Body"],
      "Tags" : (row["Tags"] if row["Tags"]!= 'non' else '') #avoiding error for null inputs
  }
  
  es.index(index="questions",id=i,document=doc)

Checking indexed data

In [7]:
es.indices.get(index='questions')

ObjectApiResponse({'questions': {'aliases': {}, 'mappings': {'properties': {'Body': {'type': 'text'}, 'ID': {'type': 'integer'}, 'Tags': {'type': 'text'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'questions', 'creation_date': '1667325834782', 'number_of_replicas': '1', 'uuid': 'w50QcniSSxiXmToE5TRRjQ', 'version': {'created': '8040399'}}}}})

In [8]:
res = es.get(index="questions", id=0)
res

ObjectApiResponse({'_index': 'questions', '_id': '0', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'ID': 62969917, 'Body': '<p>All of a sudden, I am getting this build error in my Android project:</p>\n<pre><code>unexpected element &lt;queries&gt; found in &lt;manifest&gt;\n</code></pre>\n<p>How do I fix it?</p>\n', 'Tags': '<android><android-gradle-plugin><android-manifest><manifest>'}})

Assignment(1)

In [9]:
def search_text(text):
    results = es.search(index='questions',query={"match_phrase":{"Body":text}})
    all_hits = results['hits']['hits']

    # see how many "hits" it returned using the len() function
    print ("total hits :", results['hits']['total']['value'], "\n")
    
    # iterate the nested dictionaries inside the ["hits"]["hits"] list and print results
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "\n", doc['_source']['Body'], "\n")

In [10]:
search_text('Android 12')

total hits : 2 

DOC ID: 20 
 <h2>Background</h2>
<p>In the past, I used a foreground <a href="https://developer.android.com/reference/android/app/IntentService" rel="noreferrer"><strong>IntentService</strong></a> to handle various events that come one after another. Then it was deprecated when Android 11 came (Android R, API 30) and it was said to prefer to use Worker that uses <a href="https://developer.android.com/reference/androidx/work/ListenableWorker#setForegroundAsync(androidx.work.ForegroundInfo)" rel="noreferrer"><strong>setForegroundAsync</strong></a> instead, and so I did it.</p>
<pre><code>val builder = NotificationCompat.Builder(context,...)...
setForegroundAsync(ForegroundInfo(notificationId, builder.build()))
</code></pre>
<h2>The problem</h2>
<p>As Android 12 came (Android S, API 31), just one version after , now <code>setForegroundAsync</code> is marked as deprecated, and I am told to use <a href="https://developer.android.com/reference/androidx/work/WorkRequest.Build

In [11]:
search_text('Android 14')

total hits : 0 

NOT FOUND


Assignment(2)

In [23]:
def search_id(number):
    results = es.search(index='questions',query={"term":{"ID":number}})
    all_hits = results['hits']['hits']
    print("number of results = ",results['hits']['total']['value'],"\n")
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "\n","Post ID:",doc['_source']['ID'],"\n" ,doc['_source']['Body'], "\n")

In [24]:
search_id(64034156)

number of results =  1 

DOC ID: 96 
 Post ID: 64034156 
 <p><em><strong>Short problem description:</strong></em> I had created two new AWS EC2 server instances on Redhat Linux and Ubuntu operating system. I was trying to login to it through SSH via Putty but was getting errors(detailed errors further in question). This is just a short description.</p>
<p><em><strong>Detailed problem description:</strong></em> I generated a new .pem file after creating a free tier Amazon EC2(Elastic Compute Cloud) server of Ubuntu 20 O.S version. Then I have used <code>PuttyGen</code> software in my Windows-10 O.S p.c. While trying to login through putty it was giving me an error as shown in the following error description, due to this I was not able to login to aws machines, I have further investigated the issue and all details of troubleshooting steps are mentioned in detailed manner in this self answered question(other suggestions/solutions are welcome).</p>
<p><em><strong>Error's description:</stro

In [25]:
search_id(64034157)

number of results =  0 

NOT FOUND


In [32]:
sum(df_q['Id.1']==64034156)

1

In [33]:
sum(df_q['Id.1']==64034157)

0

Assignment(3)

In [53]:
def tokenizer(tags):
    tags = tags.replace('<',"")
    tags = tags.replace('>'," ")
    return tags

In [57]:
#example
tokenizer('<python><selenium><google-chrome><selenium-chromedriver><webusb> ')

'python selenium google-chrome selenium-chromedriver webusb  '

In [59]:
# i is the doc id which starts from zero
i=0
try:
    while(True):
        tags = es.get(index='questions',id=i)['_source']['Tags']
        tokenized_tags = tokenizer(tags)
    
        res = es.search(index='questions',query={"bool": {"must_not": {"match": {"Body":tokenized_tags }},"filter":{"ids":{ "values": [i] }}}})
        value = res['hits']['total']['value'] # number of hits
        if value :
            res = es.get(index='questions',id=i)['_source']
            print('doc id : ',i,'-----','question ID :',res['ID'])
            print ('Tags : ', res['Tags'], "\n",res['Body'])
        i+=1
except:
    print('done')

doc id :  27 ----- question ID : 63075418
Tags :  <ios><uitableview><ios14> 
 <p>I'm watching and rewatching WWDC 2020 &quot;Modern cell configuration&quot; but enlightening does not strike.</p>
<p>I understand that the built-in ListContentConfiguration has properties similar to the <em>built-in cell style</em> components like the text label and the image view. But I never use the built-in cell styles; I always create a cell subclass and design the cell subviews from scratch in a xib or storyboard prototype cell, or even structure them in code.</p>
<p>So how do I use Apple's configurations to populate my cell's subviews?</p>
<p>The video says:</p>
<blockquote>
<p>In addition to the list content configuration we're also giving you access to the associated list content view which implements all of the rendering. You just create or update this view using the configuration, and then you can add it as a subview right alongside your own custom views. This lets you take advantage of all the c

PART 2 : 

In [60]:
import pandas as pd
df_a = pd.read_csv('C:\\Users\\A\\Music\\repos\\Information-Retrieval\\data\\answers.csv').fillna("")

In [63]:
mapping = {
     "properties": {
            "ParentId": {"type": "integer"},
            "CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"},
            "Score": {"type": "integer"},
            "Body": {"type": "text"},
         }}

es.indices.create(index="answers",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'answers'})

In [64]:
for i,row in df_a.iterrows():
  doc = {
      "ParentId" : row['ParentId'],
      "CreationDate" : row["CreationDate"],
      "Score" : row["Score"],
      "Body" : row["Body"],
  }
  es.index(index="answers",id=i,document=doc)

assignment 4

In [70]:
def search_Id_Lowscore(id,score):
    results = es.search(index='answers',query={"bool": {"must": [{"term": {"ParentId": id}}],"filter":{"range":{"Score": {"gte": score}}}}})
    all_hits = results['hits']['hits']
    print(results['hits']['total']['value'],' results \n')
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "--->","ParentId:", doc['_source']['ParentId'], "--->","Score = ",doc['_source']['Score'],"\n" ,doc['_source']['Body'], "\n")

In [74]:
search_Id_Lowscore(62496779,-100)

13  results 

DOC ID: 3 ---> ParentId: 62496779 ---> Score =  36 
 <p>Currently you cannot open an elevated <code>wt.exe</code> session from the command line without workarounds. Workarounds include using <a href="https://github.com/microsoft/terminal/issues/632#issuecomment-616740964" rel="noreferrer">gsudo</a>, Using <a href="https://github.com/microsoft/terminal/issues/632#issuecomment-616740964" rel="noreferrer">Task Scheduler</a> (I tested this one and it works but you need to use the full path to wt.exe and you can skip the shortcut creation step) OR if you are ok with a keyboard shortcut, the simplest way; using a keyboard shortcut to run Windows Terminal as Admin from the taskbar.</p>
<p>For your use case:</p>
<blockquote>
<p>For my specific instance, I simply want to make it simpler to pop open
an admin terminal, I don't need a way to elevate arbitrary commands,
then I will happily use the commands I have already shown here.</p>
</blockquote>
<p>The simplest approach will work

In [76]:
search_Id_Lowscore(62496779,15)

3  results 

DOC ID: 3 ---> ParentId: 62496779 ---> Score =  36 
 <p>Currently you cannot open an elevated <code>wt.exe</code> session from the command line without workarounds. Workarounds include using <a href="https://github.com/microsoft/terminal/issues/632#issuecomment-616740964" rel="noreferrer">gsudo</a>, Using <a href="https://github.com/microsoft/terminal/issues/632#issuecomment-616740964" rel="noreferrer">Task Scheduler</a> (I tested this one and it works but you need to use the full path to wt.exe and you can skip the shortcut creation step) OR if you are ok with a keyboard shortcut, the simplest way; using a keyboard shortcut to run Windows Terminal as Admin from the taskbar.</p>
<p>For your use case:</p>
<blockquote>
<p>For my specific instance, I simply want to make it simpler to pop open
an admin terminal, I don't need a way to elevate arbitrary commands,
then I will happily use the commands I have already shown here.</p>
</blockquote>
<p>The simplest approach will work:

assignment 5

In [78]:
def search_text_answers(text):
    results = es.search(index='answers',query={"match_phrase":{"Body":text}})
    all_hits = results['hits']['hits']
    print(results['hits']['total']['value'],' results \n')
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "--->",doc['_source']['Body'], "\n")

In [84]:
search_text_answers('Windows Terminal')

10  results 

DOC ID: 5 ---> <p>Try this:</p>
<pre><code>powershell &quot;Start-Process -Verb RunAs cmd.exe '/c start wt.exe'&quot;
</code></pre>
<p>Also check out these links:</p>
<p>WT.exe command line arguments:
<a href="https://learn.microsoft.com/en-us/windows/terminal/command-line-arguments?tabs=windows" rel="noreferrer">https://learn.microsoft.com/en-us/windows/terminal/command-line-arguments?tabs=windows</a></p>
<p>Article about adding Open Windows Terminal Command Prompt to the context menu in Explorer (includes Admin):
<a href="https://dkcool.tailnet.net/2020/07/add-open-windows-terminal-command-prompt-to-the-explorer-context-menu-in-windows-10/" rel="noreferrer">https://dkcool.tailnet.net/2020/07/add-open-windows-terminal-command-prompt-to-the-explorer-context-menu-in-windows-10/</a></p>
<p>Article about adding Open Admin Command Prompt to the context menu in Explorer:
<a href="https://dkcool.tailnet.net/2019/05/add-open-admin-command-prompt-to-the-explorer-context-menu-in-w

In [83]:
search_text_answers('Windows')

18  results 

DOC ID: 5 ---> <p>Try this:</p>
<pre><code>powershell &quot;Start-Process -Verb RunAs cmd.exe '/c start wt.exe'&quot;
</code></pre>
<p>Also check out these links:</p>
<p>WT.exe command line arguments:
<a href="https://learn.microsoft.com/en-us/windows/terminal/command-line-arguments?tabs=windows" rel="noreferrer">https://learn.microsoft.com/en-us/windows/terminal/command-line-arguments?tabs=windows</a></p>
<p>Article about adding Open Windows Terminal Command Prompt to the context menu in Explorer (includes Admin):
<a href="https://dkcool.tailnet.net/2020/07/add-open-windows-terminal-command-prompt-to-the-explorer-context-menu-in-windows-10/" rel="noreferrer">https://dkcool.tailnet.net/2020/07/add-open-windows-terminal-command-prompt-to-the-explorer-context-menu-in-windows-10/</a></p>
<p>Article about adding Open Admin Command Prompt to the context menu in Explorer:
<a href="https://dkcool.tailnet.net/2019/05/add-open-admin-command-prompt-to-the-explorer-context-menu-in-w

assignment 6

In [85]:
def search_answers_date(text,gte_date,lte_date):
    results = es.search(index='answers',query={"bool": {"must": [{"match_phrase": {"Body":text}}],"filter":{"range":{"CreationDate": {"gte": gte_date,"lte": lte_date}}}}})
    all_hits = results['hits']['hits']
    print(results['hits']['total']['value'],' results \n')
    if results['hits']['total']['value'] == 0:
        print('NOT FOUND')  
    else:
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "--->","CreationDate = ",doc['_source']['CreationDate'],"\n" ,doc['_source']['Body'], "\n")

In [90]:
search_text_answers("python")

7  results 

DOC ID: 189 ---> <p>Make sure u have installed a correct python version.</p>
<p>I have noted that it works on python version 3.7 or lower version.</p>
 

DOC ID: 159 ---> <p>The <code>-&gt;</code> doesn't really do much in python. </p>

<pre><code>&gt;&gt;&gt; def foo()-&gt;int:
...     return "yeet"
...
&gt;&gt;&gt; foo()
'yeet'
</code></pre>

<p>Similarly with denoting types in method signatures:</p>

<pre><code>&gt;&gt;&gt; def bar(a: int):
...     print(a)
...
&gt;&gt;&gt; bar("yeet")
'yeet'
</code></pre>

<p>The syntax is really just a means of commenting your code. Conceptually you could implement something of an interface by raising an exception in the constructor.</p>

<pre><code>&gt;&gt;&gt; class foo:
...     def __init__(self):
...             raise()
...     def someFunc(self):
...             return(42)
...
&gt;&gt;&gt; class bar(foo):
...     def __init__(self):
...             self.whatEver = "idk"
...
&gt;&gt;&gt; a = bar()
&gt;&gt;&gt; a.someFunc()
42
&gt;

In [91]:
search_answers_date("python","2020-01-01 00:00:00","2022-12-27 23:59:59")

7  results 

DOC ID: 189 ---> CreationDate =  2022-06-26 10:35:54 
 <p>Make sure u have installed a correct python version.</p>
<p>I have noted that it works on python version 3.7 or lower version.</p>
 

DOC ID: 159 ---> CreationDate =  2020-03-20 00:15:38 
 <p>The <code>-&gt;</code> doesn't really do much in python. </p>

<pre><code>&gt;&gt;&gt; def foo()-&gt;int:
...     return "yeet"
...
&gt;&gt;&gt; foo()
'yeet'
</code></pre>

<p>Similarly with denoting types in method signatures:</p>

<pre><code>&gt;&gt;&gt; def bar(a: int):
...     print(a)
...
&gt;&gt;&gt; bar("yeet")
'yeet'
</code></pre>

<p>The syntax is really just a means of commenting your code. Conceptually you could implement something of an interface by raising an exception in the constructor.</p>

<pre><code>&gt;&gt;&gt; class foo:
...     def __init__(self):
...             raise()
...     def someFunc(self):
...             return(42)
...
&gt;&gt;&gt; class bar(foo):
...     def __init__(self):
...             self.w

In [92]:
search_answers_date("python","2021-01-01 00:00:00","2022-12-27 23:59:59")

3  results 

DOC ID: 189 ---> CreationDate =  2022-06-26 10:35:54 
 <p>Make sure u have installed a correct python version.</p>
<p>I have noted that it works on python version 3.7 or lower version.</p>
 

DOC ID: 184 ---> CreationDate =  2021-12-04 10:31:14 
 <p>This works for me in all python version.</p>
<pre><code>pip install webdriver-manager
</code></pre>
<p>Here's Lib, Check out:
<a href="https://pypi.org/project/webdriver-manager/" rel="nofollow noreferrer">https://pypi.org/project/webdriver-manager/</a></p>
 

DOC ID: 186 ---> CreationDate =  2021-12-16 03:03:04 
 <p>some times just restarting your code editor can be the ultimate solution
that is what worked for me after spending 2 hours on the web trying to find out why the webdriver_manager is not picked up by python</p>
<p>use this to install</p>
<pre><code>pip install webdriver_manager
</code></pre>
<p>and then restart your VScode if that is what you are using as a code editor
I don't know why I have to but it worked the ti

PART 3

In [93]:
import pandas as pd
df_c = pd.read_csv('C:\\Users\\A\\Music\\repos\\Information-Retrieval\\data\\comments.csv').fillna("")

In [95]:
mapping = {
     "properties": {
            "Id": {"type": "integer"},
            "PostId": {"type": "integer"},
            "Body": {"type": "text"},
         }}

es.indices.create(index="comments",mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'comments'})

In [96]:
for i,row in df_c.iterrows():
    doc = {
        "Id" : row['Id'],
        "PostId" : row['PostId'],
        "Body" : row["Text"]
    }
    es.index(index="comments",id=i,document=doc)
    

In [97]:
es.get(index='comments',id=0)

ObjectApiResponse({'_index': 'comments', '_id': '0', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'Id': 105375001, 'PostId': 59603086, 'Body': 'The question is answered in [Is is valid to construct std::set of pointer type?](https://stackoverflow.com/questions/53927571/is-is-valid-to-construct-stdset-of-pointer-type)'}})

assignment 7


In [98]:
# i is the doc id which starts from zero
i=0
try:
    while(True):
        ID = es.get(index='questions',id=i)['_source']['ID']     # id of the question
        tags = es.get(index='questions',id=i)['_source']['Tags'] # tags of the question
        tokenized_tags = tokenizer(tags)

        # finding comments of the question
        res_comments = es.search(index='comments',query={"bool": {"must": {"match": {"PostId":ID}}}})
        number_comments = res_comments['hits']['total']['value'] # number of comments for the question
        for j in range(number_comments):
            # we have to define "size" because size_limit is 10 by default and does not show more than that
            comment_doc_id = es.search(index='comments',size=1000,query={"bool": {"must": {"match": {"PostId":ID}}}})['hits']['hits'][j]['_id']
            res_tag = es.search(index='comments',query={"bool": {"must": {"match": {"Body":tokenized_tags }},"filter":{"ids":{ "values": [comment_doc_id] }}}})
            value_tag = res_tag['hits']['total']['value'] # number of hits
            if value_tag :
                print('doc id : ',i,'-----','question ID :',ID)
                break
        i+=1
except:
    print('DONE')

doc id :  0 ----- question ID : 62969917
doc id :  1 ----- question ID : 65648897
doc id :  3 ----- question ID : 60114639
doc id :  5 ----- question ID : 60150749
doc id :  6 ----- question ID : 66497269
doc id :  7 ----- question ID : 60079472
doc id :  10 ----- question ID : 64081701
doc id :  11 ----- question ID : 64977541
doc id :  12 ----- question ID : 63705739
doc id :  13 ----- question ID : 63778276
doc id :  14 ----- question ID : 59869486
doc id :  15 ----- question ID : 61122378
doc id :  16 ----- question ID : 69116668
doc id :  18 ----- question ID : 61531317
doc id :  19 ----- question ID : 68332664
doc id :  20 ----- question ID : 67749284
doc id :  21 ----- question ID : 69530502
doc id :  22 ----- question ID : 67081496
doc id :  23 ----- question ID : 60442205
doc id :  26 ----- question ID : 62550828
doc id :  28 ----- question ID : 63421086
doc id :  29 ----- question ID : 65967690
doc id :  31 ----- question ID : 61521819
doc id :  32 ----- question ID : 6058304

TEST

In [102]:
#id=24 not in results
#getting Post ID
es.get(index='questions',id=24)['_source']['ID']

61536203

In [103]:
#getting tags
es.get(index='questions',id=24)['_source']['Tags']

'<javascript><node.js><firebase-realtime-database>'

In [104]:
#getting comment which is related to that post id and check tags inside the context
es.search(index='comments',query={"bool": {"must": {"match": {"PostId":61536203}}}})

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'comments', '_id': '41', '_score': 1.0, '_source': {'Id': 109971099, 'PostId': 61536203, 'Body': 'Does the same rules apply to firestore too? Having increment call in a write batch assures the data is written without any problems? (Assuming 1write/sec/per doc limit)'}}]}})

In [105]:
#id=0 is in results
es.get(index='questions',id=0)['_source']['ID']

62969917

In [106]:
es.get(index='questions',id=0)['_source']['Tags']

'<android><android-gradle-plugin><android-manifest><manifest>'

In [107]:
es.search(index='comments',query={"bool": {"must": {"match": {"PostId":62969917}}}})

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'comments', '_id': '548', '_score': 1.0, '_source': {'Id': 130428881, 'PostId': 62969917, 'Body': 'yes it is working in case <query. tag in manifest classpath("com.android.tools.build:gradle:3.5.3")\n\nCurrent setting:\n\nclasspath("com.android.tools.build:gradle:3.5.4")'}}]}})